# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [95]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

**Saving problem for a individual:**
$$
\max_{c_{1t},c_{2t+1}} \ln(c_{1t}) + \frac{1}{1+\rho} \ln(c_{2t+1})
$$

**Budget constraints:**
$$
c_{1t} + s_t = w_t (1-\tau)
$$

$$
c_{2t+1} = s_t (1+r_{t+1}) + \tau w_{t+1}
$$

**Euler equation:**
$$
c_{2t+1} = \frac{1+r_{t+1}}{1+\rho}c_{1t}
$$

**Replacing $c_{1t}$ and $c_{2t+1}$ from the budget constraints in the Euler equaiton:**
$$
s_t = \frac{1}{2+\rho} w_t (1-\tau) - \tau \frac{1+\rho}{2+\rho} \frac{1}{1+r_{t+1}} w_{t+1}
$$
where $K_{t+1} = S_t \leftrightarrow k_{t+1} = \frac{s_t}{1+n}$ and $n=0$ due to no population growth.

$$
k_{t+1} = \frac{1}{2+\rho} w_t (1-\tau) - \tau \frac{1+\rho}{2+\rho} \frac{1}{1+r_{t+1}} w_{t+1}
$$

We find $w_t$:
$$
0 = (1-\alpha) A K_t^\alpha L_{t}^{-\alpha} - w_t \leftrightarrow w_t = (1-\alpha) A K_t^\alpha L_{t}^{-\alpha} \leftrightarrow w_t = A(\frac{K_t}{L_t})^\alpha \leftrightarrow w_t = Ak_t^\alpha
$$

By inserting $w_t$ in $k_{t+1}$ we find that the **capital accumulation** is given by: 
$$
k_{t+1} = \frac{1}{2+\rho} w_t (1-\tau) - \tau \frac{1+\rho}{2+\rho} \frac{1}{1+r_{t+1}} Ak_{t+1}^\alpha
$$





In [96]:
s = sm.symbols('s')
rho = sm.symbols('rho')
w = sm.symbols('w')
tau = sm.symbols('tau')
rt1 = sm.symbols('rt1')
wt1 = sm.symbols('wt1')
ct1 = sm.symbols('ct1')
ct2 = sm.symbols('ct2')
A = sm.symbols('A')
alpha = sm.symbols('alpha')
k = sm.symbols('k')

In [ ]:
#func = lambda x: x**alpha 

steady_state = sm.Eq(k, (1/(2+rho))*w*(1-tau)-tau*((1+rho)/(2+rho))*(1/(1+rt1))*A*k**alpha)
ks = sm.solve(steady_state, k)[0]
ks

# Man kan ikke løse for kt1 analytisk, så vi gør det numerisk 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [ ]:
rho = 0.2 # skal ligge mellem 0 og 1
w = 1
tau = 0.4 # skal ligge mellem 0 og 1
rt1 = 0.5
A = 1
alpha = 0.5 # skal ligge mellem 0 og 1

obj = lambda k: k - (1/(2+rho))*w*(1-tau)-tau*((1+rho)/(2+rho))*(1/(1+rt1))*A*k**alpha

result = optimize.root_scalar(obj,bracket=[1e-10,10],method='brentq') 
result.root

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # vi kan ikke have alpha = 1.0, da nævneren derved bliver 0 

for alpha in alphas: 
    k_new = lambda x: (1/(2+rho)*(1-alpha)*A)**(1/(1-alpha))
    obj1 = lambda k_ny: k_ny - (1/(2+rho))*w*(1-tau)-tau*((1+rho)/(2+rho))*(1/(1+rt1))*A*k_new(k_ny)
    res = optimize.root_scalar(obj1,bracket=[1e-10,10],method='brentq')
    print(res.root)

# Conclusion

Add concise conclusion. 